# Anchor explanations on the Iris dataset

In [ ]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.ensemble import RandomForestClassifier
from alibi.explainers import AnchorTabular

### Load iris dataset

In [ ]:
dataset = load_iris()
feature_names = dataset.feature_names
class_names = list(dataset.target_names)

Define training and test set

In [ ]:
idx = 145
X_train,Y_train = dataset.data[:idx,:], dataset.target[:idx]
X_test, Y_test = dataset.data[idx+1:,:], dataset.target[idx+1:]

### Train Random Forest model

In [ ]:
np.random.seed(0)
clf = RandomForestClassifier(n_estimators=50)
clf.fit(X_train, Y_train)

Define predict function

In [ ]:
predict_fn = lambda x: clf.predict_proba(x)

### Initialize and fit anchor explainer for tabular data

In [ ]:
explainer = AnchorTabular(predict_fn, feature_names)

Discretize the ordinal features into quartiles

In [ ]:
explainer.fit(X_train, discretizer='quartile')

### Getting an anchor

Below, we get an anchor for the prediction of the first observation in the test set. An anchor is a sufficient condition - that is, when the anchor holds, the prediction should be the same as the prediction for this instance.

In [ ]:
idx = 0
print('Prediction: ', class_names[explainer.predict_fn(X_test[idx].reshape(1, -1))[0]])

We set the precision threshold to 0.95. This means that predictions on observations where the anchor holds will be the same as the prediction on the explained instance at least 95% of the time.

In [ ]:
explanation = explainer.explain(X_test[idx], threshold=0.95)
print('Anchor: %s' % (' AND '.join(explanation['names'])))
print('Precision: %.2f' % explanation['precision'])
print('Coverage: %.2f' % explanation['coverage'])